In [1]:
from keras.callbacks import LambdaCallback # 학습률을 표시해주는 callback 함수
from keras.models import Sequential # 케라스 모델
from keras.layers import Dense, LSTM #딥 러닝용 레이어
from tensorflow.keras.optimizers import Adam, RMSprop #옵티마이저,손실 함수를 기반으로 네트워크가 어떻게 업데이트될지 결정합니다. 특정 종류의 확률적 경사 하강법(SGD)을 구현합니다.(에러1 해결)
from keras.utils.data_utils import get_file #케라스 util

import numpy as np

import random
import sys
import io
import re

Preprocess

In [2]:
# path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

path = './song_folder/lyric_data_hiphop.txt'
with io.open(path, encoding='utf-8') as f: #텍스트 파일을 f에 읽어옴 (한국어 인코딩인 CP949사용 )
    text = f.read().lower() #f에 내용들을 소문자화

text = re.sub(r'\n', ' ', text) #줄바꿈을 공백으로
text = re.sub(r'-', ' ', text) # '-'를 공백으로
text = ' '.join(text.split()) #여러개의 공백을 제거하고 하나의 문자열로 만듦

print('corpus length:', len(text)) 
print(text) #전처리한 text 확인

corpus length: 1531912
하얀색 도화지 누가 더 특별한지로 다 다퉈 검은색 화폭 빨간 선이나 긋지 나 좀 놔줘 순수한 삶도 검기만 했던 내 밤도 노랗디 노란 내 달도 나로 살아가게 놔둬 나는 나로 죽게 놔둬 내 철학 느긋해 그리고 철저 그래서 내가 누군데 나의 죽음 뒤에 뭐가 따라오든 난 그냥 나로 죽을래 나 빼고 묵음 yeah 이것저것 구분없이 다 묶음 가끔 난 두부처럼 부셔지다가도 단단해져 그래서 내가 대체 누군데 그래서 내가 대체 누군데 그래서 내가 대체 누군데 그래서 내가 대체 누군데 누군데 내가 대체 누군데 누군데 누군데 이 새끼는 누군데 2년 전과의 누구와는 달라 갈아 끼운 부품 yeah 너희 오빤 잡아 까락 그리고 외상이나 달아 비슷한 찍어내기 그런 건 나 역시도 필요 없네 야밤에 야밤에 나 혼자서 포커페이스 너의 그 조급함이 목 졸라 언더테이커 초코슬램 야심한 밤에 넌 전화해도 돼 가끔은 좋더라 그런 뻔한 게 그런 뻔한 게 뻔하고 뻔한 게 되면 난 쿨하게 널 보내 이 여유를 main tain 조급함이 개입되기 전에 얼음땡 하고 game set 누가 제일 쎈 놈인지 밝히는 게임에 나는 잽잽을 누적하다가도 명존쎄 이 새끼는 누군데 너가 안될 거라 했던 얘는 대체 누군데 choo 누군데 그래서 내가 대체 누군데 그래서 내가 대체 누군데 그래서 내가 대체 누군데 누군데 내가 대체 누군데 검은 날개를 펴 나비 돼 이제 퍼런 별 번쩍 뜨는 밤이 돼있네 난 아직 깨있네 난 아직 깨있네 이 거리 담 넘어서라도 난 가질 게 있네 나 빼고 이제 다 잃을 각오로도 맘 한켠에 지키려는 건 무게 이걸 무시하면 다이렉트로다 니 눈 밑에 찍 그어지는 원의 둘레 난 그냥 지르고서 후에야 수습하는 스타일 난 매 순간을 나로 살다 죽을래 imma die me 달리고 벌려 차이 투명한 척 이건 마임 난 아직도 투쟁에 살고 이건 영원한 타입 imma die me 난 내가 뿌린 조각을 밟고 와있지 잡아 타이밍 너도 fighting 난 내가 뿌린 조각을 밟고 와있지 누

# Compute Character Indices

In [5]:
# 문자를 하나하나 읽어 들이고 ID 붙이기
chars = sorted(list(set(text))) #단어들을 정렬하여 리스트화
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars)) # 문자 → ID
indices_char = dict((i, c) for i, c in enumerate(chars)) # ID → 문자

total chars: 1977


# Vectorize Sentences

In [6]:
# 텍스트를 maxlen개의 문자로 자르고 다음에 오는 문자 등록하기
maxlen = 25
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen]) # i~i+39 번째 단어까지 저장
    next_chars.append(text[i + maxlen]) # i+40번째 단어 저장
print('학습할 구문의 수:', len(sentences)) #학습할 구문의 수

print('Vectorization...') # 텍스트를 벡터로 변환
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool) #sentencs by 40 by 1608 zero matrix 생성
y = np.zeros((len(sentences), len(chars)), dtype=np.bool) #sentencs by 1608 zero matrix 생성

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1 # 텍스트의 id 값을 가지고 있는 matrix에 1을 넣음
    y[i, char_indices[next_chars[i]]] = 1 # 다음텍스트의 id값을 가지고 있는 matrix에 1을 넣음

학습할 구문의 수: 174918
Vectorization...


C:\Users\tkana\AppData\Local\Temp/ipykernel_9032/2926439413.py:13: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool) #sentencs by 40 by 1608 zero matrix 생성
C:\Users\tkana\AppData\Local\Temp/ipykernel_9032/2926439413.py:14: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(s

# Create Simple Model

In [7]:
print('Build model...')
#keras에서는 모델을 만드는데 두 가지 방법이 있는데 
#Sequential 모델은 순차적으로 레이어 층을 더해주기 때문에 
#순차모델이라 불리고 만들기도 쉽다.

model = Sequential() #모델을 순차 모델로 만든다.
model.add(LSTM(128, input_shape=(maxlen, len(chars)))) # LSTM #40 x 1608 짜리, 입력
model.add(Dense(len(chars), activation='softmax')) # Dense #1608 짜리, 출력

model.compile(optimizer=RMSprop(lr=0.01), loss='categorical_crossentropy') #모델 컴파일

Build model...


C:\Users\tkana\AppData\Roaming\Python\Python39\site-packages\keras\optimizer_v2\rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


# Define Callback

In [8]:
def sample(preds, temperature=1.0):#약간의 랜덤성을 주기 위해 확률을 조작 (약간의 돌발성 생성)
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# 학습시키고 텍스트 생성하기 반복
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('반복 =', iteration)
    model.fit(x, y, epochs=1) # 
    # 임의의 시작 텍스트 선택하기
    start_index = random.randint(0, len(text) - maxlen - 1)
    # 다양한 다양성의 문장 생성
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('--- 다양성 = ', diversity)
        generated = ''
        sentence = '밤'
        generated += sentence
        print('--- 시드 = "' + sentence + '"')
        sys.stdout.write(generated)
        # 시드를 기반으로 텍스트 자동 생성
        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.
            # 다음에 올 문자를 예측하기
            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            # 출력하기
            generated += next_char
            sentence = sentence[1:] + next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


'''
def on_epoch_end(epoch, _):
    print('\n----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1) #40개의 글자를 랜덤으로 뽑아 모델에게 힌트로 주고 그 뒷 부분을 지어내게 함 
#     for diversity in [0.2, 0.5, 1.0, 1.2]:
#         print('----- diversity:', diversity)

#     for diversity in [0.2, 0.5, 1.0, 1.2]:
#         print('----- diversity:', diversity)

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(400): #에폭이 끝날 때마다 약간의 힌트(sentence 변수)를 주고 그 뒤의 400 글자 지어보게 하기
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, 0.5)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end) #커스텀 콜백
'''


--------------------------------------------------
반복 = 1
5467/5467 [==============================] - 542s 99ms/step - loss: 3.4986

--- 다양성 =  0.2
--- 시드 = "밤"
밤     지       아  지             아   지      어          지 아      지                             아지                      이               이     기 다  지                            라      아   아         지                                 하고       지지                          다  아          지 지   지          다   지 이              이      지          해            지      도지           이                        지        

--- 다양성 =  0.5
--- 시드 = "밤"
밤 거 나어했가 어이이  해하이   아지시 가 하는을이다려 도  지  지  가 한  이전 은도 아 던시 아 을   기 을 는아수 아가다아 아도아거마 은 이하인할은 지 지지  지  가도아다이에하하 아아하 어보가해지지이지지던은  지하지 지지는지아 아했지 라가다아다네 어지 은 지 도가  도기나   지도   아아하 이는아 라지아 기     도  어지거이과지 나아지시려마   다지 아마지   다게어거하 아하  지네한지 지어에이하니 으다 는   지가해 지도 지지 보지 어지  한기네  나아하버    한네이아 에아가했 이어기야 에이비다제가 지 기다 아다 지기아어   이 어   아다  치은   지가 한지 이는게 지이 지아이어이시 가지 할 보가   지보 가 고 지도 지a다 한기 려  마있나다다하지 해한지아가이  아다 나 나해은어

---

ValueError: Data cardinality is ambiguous:
  x sizes: 1
  y sizes: 174918
Make sure all arrays contain the same number of samples.

# Train

In [47]:
model.fit(x, y, batch_size=128, epochs=60, callbacks=[print_callback])

Epoch 1/60
602/602 [==============================] - ETA: 0s - loss: 2.6226
----- Generating text after Epoch: 0
----- Generating with seed: "     "
                                                       나 나를 몰라서 지금 않아 영원히 기억해 좋은한 거야 속 이제 어껴져 가도 없이 될 까 미소 맘은 그때 우리 함께  자 있을 거야 알아 걸어 그대를 보고 싶어 너와 나워지도 않아 다시 그대로 맘 걸어가 없는 걸 알고 있을까 그 날 언제나 나는 모든 순간이 나도 그래도 나는 나의 모든 순간이 떠올려 그대의 모든 날이 모든 걸 이제는 내 맘이 너무 난 괜찮아요 우리의 함께 다시 내게 해 너를 사랑하는 걸 알지 않아 또 날 바라보본다 너와 나나 나나의 마음이 나는 그대로 해 차가 없어 내 맘이 떠난간이 내가 보고 싶어요 오늘도 나는 모든 사랑이 아닌 수 없었어 쯤 한 것도 너를 보고 있을까 우리 사랑한 내 맘이 너무 멈춰줄게
602/602 [==============================] - 261s 433ms/step - loss: 2.6226
Epoch 2/60
602/602 [==============================] - ETA: 0s - loss: 2.4127
----- Generating text after Epoch: 1
----- Generating with seed: "일이 처음"
일이 처음 그대 눈물이 나는 나를 찾아 보아 내가 나는 이젠 아직 너의 사랑을                                                                                                                                                                                 

Epoch 14/60
602/602 [==============================] - ETA: 0s - loss: 0.9029
----- Generating text after Epoch: 13
----- Generating with seed: "지 매일 "
지 매일 밤 마다 그대가 부담요 내 맘을 알고줘 사랑 그 사랑을 그 다 지울 수 있을까 사랑한다 너 없는 하루가 길 가을운 오늘 밤 그와 그리면 지금 이대로 아이 남아있음 많다 걸을 가면 그 언젠가 다시 우리가 잊어볼아 그 말을 꺼지면 stay to gel ill stay we den’ lest i ceull never eves on my love 이렇게 우리가 만나던 날 사랑 한 기큼 속에 네 마음을 지나 그대도 나처럼 눈물에 다르지 않아 지금 그대 다시 돌아올 거면 나는 너를 향한 나의 마음을 지나 그대도 날 생각해 난 여기라아 이 부르는 meor i love i’m ald all oh my love 세상 전한 내 마음 알디주기 하고 어느 향한 눈물이 흘러도 눈물 없죠 어떻게 해야 하는지 몰라 한마가 지워야 하는
602/602 [==============================] - 187s 311ms/step - loss: 0.9029
Epoch 15/60
602/602 [==============================] - ETA: 0s - loss: 0.8529
----- Generating text after Epoch: 14
----- Generating with seed: "     "
                                                                                                                                                                                                                        

KeyboardInterrupt: 